Transformer

In [1]:
#test
import torch

print(torch.__version__)
torch.cuda.is_available()

layercount = 0
print("START\n\n\n\n",file=open("output.txt", "w"))

1.10.0


In [2]:
import torch
import torch.nn as nn


class SelfAttention(nn.Module):
    def __init__(self, embed_size, heads):
        super(SelfAttention, self).__init__()
        self.embed_size = embed_size
        self.heads = heads
        self.head_dim = embed_size // heads

        assert (
            self.head_dim * heads == embed_size
        ), "Embedding size needs to be divisible by heads"

        self.values = nn.Linear(self.head_dim, self.head_dim, bias=False)
        self.keys = nn.Linear(self.head_dim, self.head_dim, bias=False)
        self.queries = nn.Linear(self.head_dim, self.head_dim, bias=False)
        self.fc_out = nn.Linear(heads * self.head_dim, embed_size)
        
        #fix the weights
        
        #, file=open("output.txt", "a")

    def forward(self, values, keys, query, mask):
        # Get number of training examples
        N = query.shape[0]

        value_len, key_len, query_len = values.shape[1], keys.shape[1], query.shape[1]

        # Split the embedding into self.heads different pieces
        values = values.reshape(N, value_len, self.heads, self.head_dim)
        keys = keys.reshape(N, key_len, self.heads, self.head_dim)
        query = query.reshape(N, query_len, self.heads, self.head_dim)

        global layercount
        layercount += 1
        print("\n\n======================layer", layercount, "======================\n\n",file=open("output.txt", "a"))
        
        values = self.values(values)  # (N, value_len, heads, head_dim)
        #torch.set_printoptions(threshold=10_000)
        print("==============values============\n", values,file=open("output.txt", "a"))
        print(values.size(),file=open("output.txt", "a"))
        print("Values in the correct order, head wise",file=open("output.txt", "a")) # this only works in with value_len = 1
        for h in range (self.heads):
            print ("head-", h+1, "{", end = "",file=open("output.txt", "a"))
            for l in range (value_len):
                for elem in list(values[0,l,h,:].flatten()):
                    print('{:.4f}'.format(elem), end=",",file=open("output.txt", "a"))
            print("};",file=open("output.txt", "a"))
        print("};",file=open("output.txt", "a"))


        keys = self.keys(keys)  # (N, key_len, heads, head_dim)
        print("==============keys============\n", keys)
        for h in range (self.heads):
            print ("head-", h+1, "{", end = "")
            for l in range (key_len):
                for elem in list(keys[0,l,h,:].flatten()):
                    print('{:.4f}'.format(elem), end=",")
            print("};")
        print("};")
        
        queries = self.queries(query)  # (N, query_len, heads, heads_dim)
        print("==============queries============\n", queries)
        for h in range (self.heads):
            print ("head-", h+1, "{", end = "")
            for l in range (query_len):
                for elem in list(queries[0,l,h,:].flatten()):
                    print('{:.4f}'.format(elem), end=",")
            print("};")
        print("};")

        # Einsum does matrix mult. for query*keys for each training example
        # with every other training example, don't be confused by einsum
        # it's just how I like doing matrix multiplication & bmm

        energy = torch.einsum("nqhd,nkhd->nhqk", [queries, keys])
        print("================energy=============\n", energy)
        for h in range (self.heads):
            print ("head-", h+1, "{", end = "")
            for elem in list(energy[0,h,:,:].flatten()):
                print('{:.4f}'.format(elem), end=",")
            print("};")
        print("};")
        # queries shape: (N, query_len, heads, heads_dim),
        # keys shape: (N, key_len, heads, heads_dim)
        # energy: (N, heads, query_len, key_len)

        # Mask padded indices so their weights become 0
        if mask is not None:
            energy = energy.masked_fill(mask == 0, float("-1e20"))
            #print("================energy mask=============\n", energy)

        # Normalize energy values similarly to seq2seq + attention
        # so that they sum to 1. Also divide by scaling factor for
        # better stability
        attention = torch.softmax(energy / (self.embed_size ** (1 / 2)), dim=3)
        for h in range (self.heads):
            print ("head-", h+1, "{", end = "")
            for elem in list(attention[0,h,:,:].flatten()):
                print('{:.4f}'.format(elem), end=",")
            print("};")
        print("};")
        print("self.ember_size = ", self.embed_size)
        #print("\n===========attention==========\n", attention)
        # attention shape: (N, heads, query_len, key_len)
        
        
        out = torch.einsum("nhql,nlhd->nqhd", [attention, values]).reshape(
            N, query_len, self.heads * self.head_dim
        )
        print("\n===========out after concat==========\n", out)
        for l in range (query_len):
            print ("row", l+1,"of all heads concatenated " "{", end = "")
            for elem in list(out[0,l,:].flatten()):
                print('{:.4f}'.format(elem), end=",")
            print("};")
        print("};")
        # attention shape: (N, heads, query_len, key_len)
        # values shape: (N, value_len, heads, heads_dim)
        # out after matrix multiply: (N, query_len, heads, head_dim), then
        # we reshape and flatten the last two dimensions.

        out = self.fc_out(out)
        
        print("\nlast lin in MHA", self.fc_out.weight, "\n bias for last lin", self.fc_out.bias)
        
        print("\n\nWeights for lin layer [24x24] = {", end=" ")
        for elem in list(self.fc_out.weight.flatten()):
            print('{:.4f}'.format(elem), end=",")
        print("};")
        
        print("\n\nbias for lin layer [1x24] = {", end=" ")
        for elem in list(self.fc_out.bias.flatten()):
            print('{:.4f}'.format(elem), end=",")
        print("};")
        
        print("\n===========out linear layer==========\n", out)
        
        for l in range (query_len):
            print ("row", l+1,"of all heads concatenated " "{", end = "")
            for elem in list(out[0,l,:].flatten()):
                print('{:.4f}'.format(elem), end=",")
            print("};")
        print("};")
        # Linear layer doesn't modify the shape, final shape will be
        # (N, query_len, embed_size)
        
        print(out.size())

        return out


class TransformerBlock(nn.Module):
    def __init__(self, embed_size, heads, dropout, forward_expansion):
        super(TransformerBlock, self).__init__()
        self.attention = SelfAttention(embed_size, heads)
        self.norm1 = nn.LayerNorm(embed_size)
        self.norm2 = nn.LayerNorm(embed_size)

        self.feed_forward = nn.Sequential(
            nn.Linear(embed_size, forward_expansion * embed_size),
            #also fix weights
            nn.ReLU(),
            nn.Linear(forward_expansion * embed_size, embed_size),
            #fix the weights
        )

        self.dropout = nn.Dropout(dropout)

    def forward(self, value, key, query, mask):
        attention = self.attention(value, key, query, mask)
        print(attention.size())
        print(query.size())
        # Add skip connection, run through normalization and finally dropout
        x = self.dropout(self.norm1(attention + query))
        print("\n==========norm1==========\n",x)
        print(x.size())
        for l in range (query.shape[1]):
            print ("row", l+1,"of all heads concatenated " "{", end = "")
            for elem in list(x[0,l,:].flatten()):
                print('{:.4f}'.format(elem), end=",")
            print("};")
        print("};")
        
        
        forward = self.feed_forward(x)
        
        
        print("\nweights in FF", self.feed_forward[0].weight, "\n bias in FF", self.feed_forward[0].bias)
        print("\n\n weights for first layer in FF [96x24] = {", end=" ",file=open("output.txt", "a"))
        for elem in list(self.feed_forward[0].weight.flatten()):
            print('{:.4f}'.format(elem), end=",",file=open("output.txt", "a"))
        print("};",file=open("output.txt", "a"))
        
        print("\n\nbias for first layer in FF [1x96] = {", end=" ",file=open("output.txt", "a"))
        for elem in list(self.feed_forward[0].bias.flatten()):
            print('{:.4f}'.format(elem), end=",",file=open("output.txt", "a"))
        print("};",file=open("output.txt", "a"))
        
        
        print("\nweights in FF", self.feed_forward[2].weight, "\n bias in FF", self.feed_forward[2].bias)
        print("\n\n weights for second layer in FF [24x96] = {", end=" ",file=open("output.txt", "a"))
        for elem in list(self.feed_forward[2].weight.flatten()):
            print('{:.4f}'.format(elem), end=",",file=open("output.txt", "a"))
        print("};",file=open("output.txt", "a"))
        
        print("\n\nbias for second layer in FF [1x24] = {", end=" ",file=open("output.txt", "a"))
        for elem in list(self.feed_forward[2].bias.flatten()):
            print('{:.4f}'.format(elem), end=",",file=open("output.txt", "a"))
        print("};",file=open("output.txt", "a"))
        
        
        
        
        
        out = self.dropout(self.norm2(forward + x))
        
        return out


class Encoder(nn.Module):
    def __init__(
        self,
        src_vocab_size,
        embed_size,
        num_layers,
        heads,
        device,
        forward_expansion,
        dropout,
        max_length,
    ):

        super(Encoder, self).__init__()
        self.embed_size = embed_size
        self.device = device
        self.word_embedding = nn.Embedding(src_vocab_size, embed_size)
        self.position_embedding = nn.Embedding(max_length, embed_size)

        self.layers = nn.ModuleList(
            [
                TransformerBlock(
                    embed_size,
                    heads,
                    dropout=dropout,
                    forward_expansion=forward_expansion,
                )
                for _ in range(num_layers)
            ]
        )

        self.dropout = nn.Dropout(dropout)

    def forward(self, x, mask):
        N, seq_length = x.shape
        positions = torch.arange(0, seq_length).expand(N, seq_length).to(self.device)
        out = self.dropout(
            (self.word_embedding(x) + self.position_embedding(positions))
        )

        # In the Encoder the query, key, value are all the same, it's in the
        # decoder this will change. This might look a bit odd in this case.
        for layer in self.layers:
            out = layer(out, out, out, mask)

        return out

In [3]:
class DecoderBlock(nn.Module):
    def __init__(self, embed_size, heads, forward_expansion, dropout, device):
        super(DecoderBlock, self).__init__()
        self.norm = nn.LayerNorm(embed_size)
        self.attention = SelfAttention(embed_size, heads=heads)
        self.transformer_block = TransformerBlock(
            embed_size, heads, dropout, forward_expansion
        )
        self.dropout = nn.Dropout(dropout)

    def forward(self, x, value, key, src_mask, trg_mask):
        attention = self.attention(x, x, x, trg_mask)
        query = self.dropout(self.norm(attention + x))
        out = self.transformer_block(value, key, query, src_mask)
        return out


class Decoder(nn.Module):
    def __init__(
        self,
        trg_vocab_size,
        embed_size,
        num_layers,
        heads,
        forward_expansion,
        dropout,
        device,
        max_length,
    ):
        super(Decoder, self).__init__()
        self.device = device
        self.word_embedding = nn.Embedding(trg_vocab_size, embed_size)
        self.position_embedding = nn.Embedding(max_length, embed_size)

        self.layers = nn.ModuleList(
            [
                DecoderBlock(embed_size, heads, forward_expansion, dropout, device)
                for _ in range(num_layers)
            ]
        )
        self.fc_out = nn.Linear(embed_size, trg_vocab_size)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x, enc_out, src_mask, trg_mask):
        N, seq_length = x.shape
        positions = torch.arange(0, seq_length).expand(N, seq_length).to(self.device)
        x = self.dropout((self.word_embedding(x) + self.position_embedding(positions)))

        for layer in self.layers:
            x = layer(x, enc_out, enc_out, src_mask, trg_mask)

        out = self.fc_out(x)

        return out


In [4]:
class Transformer(nn.Module):
    def __init__(
        self,
        src_vocab_size,
        trg_vocab_size,
        src_pad_idx,
        trg_pad_idx,
        embed_size=24,
        num_layers=6,
        forward_expansion=4,
        heads=8,
        dropout=0,
        device="cpu",
        max_length=100,
    ):

        super(Transformer, self).__init__()

        self.encoder = Encoder(
            src_vocab_size,
            embed_size,
            num_layers,
            heads,
            device,
            forward_expansion,
            dropout,
            max_length,
        )

        self.decoder = Decoder(
            trg_vocab_size,
            embed_size,
            num_layers,
            heads,
            forward_expansion,
            dropout,
            device,
            max_length,
        )

        self.src_pad_idx = src_pad_idx
        self.trg_pad_idx = trg_pad_idx
        self.device = device

    def make_src_mask(self, src):
        src_mask = (src != self.src_pad_idx).unsqueeze(1).unsqueeze(2)
        # (N, 1, 1, src_len)
        return src_mask.to(self.device)

    def make_trg_mask(self, trg):
        N, trg_len = trg.shape
        trg_mask = torch.tril(torch.ones((trg_len, trg_len))).expand(
            N, 1, trg_len, trg_len
        )

        return trg_mask.to(self.device)

    def forward(self, src, trg):
        src_mask = self.make_src_mask(src)
        trg_mask = self.make_trg_mask(trg)
        enc_src = self.encoder(src, src_mask)
        out = self.decoder(trg, enc_src, src_mask, trg_mask)
        return out

In [5]:
if __name__ == "__main__":
    device = torch.device("cpu") #("cuda" if torch.cuda.is_available() else "cpu")
    print(device)
     
    src_pad_idx = 0
    trg_pad_idx = 0
    src_vocab_size = 10
    trg_vocab_size = 10
    model = Transformer(src_vocab_size, trg_vocab_size, src_pad_idx, trg_pad_idx, device=device).to(
        device
    )
    '''with torch.no_grad():
    model.linear1.weight.data[...] = torch.Tensor([[-0.1], [0.2]])
    '''
    model.load_state_dict(torch.load("txmodel.pt"))
    model.eval()
    
    


cpu


In [6]:
    x = torch.tensor([[1,2,3]]).to(device)
    trg = torch.tensor([[4,5,6]]).to(device)
    
    out = model(x, trg[:, :-1])
    print(out.shape)
    print(out)
    

==============keys============
 tensor([[[[ 1.0760e+00,  5.9140e-01,  9.2687e-01],
          [ 2.1221e-01,  3.4386e-01,  3.1073e-01],
          [ 7.5313e-01,  4.1671e-01,  6.8236e-01],
          [-5.7343e-01,  1.3944e-01, -1.5985e-01],
          [ 1.5469e+00, -1.0357e-02,  7.3495e-01],
          [-5.2288e-01,  1.3377e-01, -1.3494e-01],
          [-1.0442e-03, -4.3914e-02, -1.9722e-02],
          [-1.0766e+00,  6.0791e-02, -4.5844e-01]],

         [[-9.8349e-01, -1.1192e-01, -5.7491e-01],
          [-5.3356e-01, -3.6292e-02, -2.7085e-01],
          [-2.1358e-01,  3.5088e-03, -1.2214e-01],
          [-1.0191e+00, -1.0534e-01, -4.8059e-01],
          [-7.8539e-01,  1.8612e-01, -1.9893e-01],
          [ 3.1365e-01,  6.2498e-01,  6.0563e-01],
          [-3.3106e-01, -3.6228e-01, -4.8869e-01],
          [ 1.8706e-01,  5.8515e-01,  4.5563e-01]],

         [[ 2.1601e+00,  5.2096e-01,  1.4302e+00],
          [-1.1319e-02, -2.1905e-01, -2.0319e-01],
          [-2.9264e-02,  1.6058e-01,  7.4451e-

Weights for lin layer [24x24] = { 0.1856,0.0339,0.1720,0.1393,-0.1956,-0.0759,0.0913,-0.0508,0.1631,0.1151,-0.1848,-0.0831,-0.1088,-0.1389,-0.1567,-0.0373,0.1243,0.0010,0.1157,0.1482,0.0205,0.0996,0.1668,-0.1993,0.0555,-0.0214,0.0052,-0.1093,0.0618,0.1430,-0.0916,-0.0385,-0.0268,0.1304,0.1953,0.1442,-0.1767,-0.0784,0.0531,0.1612,-0.1875,0.1213,0.1294,0.1893,-0.0821,-0.1282,0.0668,-0.0970,0.1699,-0.0467,-0.0431,0.1461,-0.0869,0.0179,0.0363,0.1141,0.0287,0.1132,0.1455,0.1401,-0.0804,0.1155,-0.1493,-0.1239,0.0228,-0.1287,0.0875,-0.0889,0.0620,0.0036,-0.1673,-0.1588,0.1008,0.0876,0.1156,0.1497,-0.0409,0.0751,0.0198,-0.0787,-0.0556,0.1860,0.2000,-0.1909,-0.0153,0.0190,0.1611,0.0416,-0.0209,0.0111,-0.1304,0.1538,0.1012,0.1240,-0.0510,-0.1955,-0.1292,-0.1082,0.0230,-0.1279,0.0177,-0.1764,0.1792,-0.1876,-0.0319,-0.0584,-0.0666,0.0075,-0.1832,0.1862,-0.0025,0.1132,-0.1661,-0.1449,-0.1743,0.0833,-0.0538,0.1233,0.1086,0.0995,-0.0963,0.0613,0.0043,-0.1632,-0.0143,-0.1481,-0.0294,-0.1141,0.0884,-0.


weights in FF Parameter containing:
tensor([[ 0.0035,  0.0142,  0.0672,  ...,  0.0286,  0.0088,  0.0510],
        [-0.0338,  0.0274, -0.0770,  ..., -0.0418,  0.0223, -0.0525],
        [-0.0748, -0.1006, -0.0133,  ...,  0.0038,  0.0181, -0.0230],
        ...,
        [ 0.0827,  0.0382,  0.0295,  ...,  0.0563,  0.0560,  0.0720],
        [ 0.0429, -0.0251, -0.0165,  ...,  0.0086,  0.0540, -0.0695],
        [ 0.0617,  0.0221,  0.0119,  ..., -0.0198,  0.0141, -0.0213]],
       requires_grad=True) 
 bias in FF Parameter containing:
tensor([ 0.0786, -0.0365, -0.0572, -0.0158, -0.0113, -0.1015,  0.0186,  0.0666,
        -0.1004, -0.0283, -0.0258,  0.1008,  0.0906, -0.0518,  0.1015, -0.0271,
        -0.0647,  0.0038, -0.0229,  0.0186,  0.0041, -0.1018, -0.0507,  0.0080],
       requires_grad=True)
==============keys============
 tensor([[[[-1.0978, -0.0345,  0.8653],
          [-0.4619,  0.5675, -0.3063],
          [-1.1682, -0.5748,  1.3453],
          [-0.5097, -0.0749,  0.0962],
          [


weights in FF Parameter containing:
tensor([[-0.0775, -0.0096, -0.0363,  ...,  0.0647,  0.0294, -0.0941],
        [ 0.0686,  0.0691,  0.0106,  ...,  0.0095, -0.0626, -0.0906],
        [-0.0493, -0.0779, -0.0424,  ..., -0.0442,  0.0959,  0.0454],
        ...,
        [-0.0865,  0.0194, -0.0621,  ..., -0.0180, -0.0469, -0.0997],
        [ 0.0067, -0.0378,  0.0150,  ..., -0.0428,  0.0388, -0.0688],
        [-0.0416, -0.0849, -0.0400,  ...,  0.0509, -0.0650, -0.0598]],
       requires_grad=True) 
 bias in FF Parameter containing:
tensor([-0.0952,  0.1001,  0.0130,  0.0275, -0.0426,  0.0888,  0.0041,  0.0420,
         0.0964, -0.0237, -0.0566, -0.0083, -0.0466,  0.0422, -0.0065, -0.0131,
        -0.0320, -0.0803, -0.0334, -0.0881,  0.0393, -0.0063, -0.0604, -0.0719],
       requires_grad=True)
==============keys============
 tensor([[[[ 0.3002,  0.3979,  1.6510],
          [-0.8640, -1.0478,  0.1554],
          [ 0.7301,  0.8367,  0.9860],
          [-0.2098, -0.3027,  0.5731],
          [


weights in FF Parameter containing:
tensor([[ 0.0838,  0.0562,  0.0238,  ...,  0.0330,  0.0915, -0.0323],
        [ 0.0361, -0.0417, -0.1009,  ..., -0.0357,  0.0012,  0.0785],
        [-0.0622,  0.0363,  0.0521,  ...,  0.0729, -0.0825, -0.0457],
        ...,
        [-0.0185,  0.0836,  0.0790,  ...,  0.0878, -0.0525, -0.0950],
        [-0.0360,  0.0704, -0.0987,  ...,  0.0568, -0.0236,  0.0015],
        [-0.0166,  0.0801,  0.0652,  ..., -0.0252, -0.0940,  0.0865]],
       requires_grad=True) 
 bias in FF Parameter containing:
tensor([ 9.9836e-02,  9.9699e-02, -4.2582e-02,  9.7536e-02,  1.7437e-02,
        -2.1030e-03, -9.7800e-02, -5.7389e-03,  9.2731e-02, -9.5670e-02,
         7.9425e-02, -4.4737e-02, -8.9243e-02,  7.3232e-05,  3.5135e-02,
        -7.6946e-02,  5.3030e-02, -7.4917e-02,  1.1119e-02, -5.2541e-02,
        -8.9631e-02,  1.0530e-02,  9.1829e-02, -4.1162e-02],
       requires_grad=True)
==============keys============
 tensor([[[[-6.5346e-01, -6.3165e-01,  1.2133e-01],
    

...

Weights for lin layer [24x24] = { 0.1745,-0.2006,-0.0176,-0.2033,-0.0605,0.1743,0.0493,-0.0095,-0.0018,0.1866,-0.0875,0.0813,-0.0046,0.0657,-0.0224,-0.0914,-0.1638,-0.0984,0.1763,0.0971,-0.0131,-0.1714,0.1959,-0.0311,-0.0024,0.0113,0.0106,0.1548,0.0535,0.1144,-0.1874,-0.0095,0.0635,-0.1850,0.0557,-0.0911,0.1292,0.1608,-0.0729,0.1068,-0.1932,0.1071,0.0726,0.0144,-0.0565,0.0057,-0.0822,-0.1409,0.0487,0.1777,0.0204,-0.1290,0.0893,-0.1389,0.1174,0.0458,0.1541,0.1999,0.1471,-0.2018,0.1121,-0.1436,0.2024,-0.1383,0.0719,0.1597,-0.1837,0.1777,0.0879,-0.0153,0.1875,0.2031,0.1101,0.1340,-0.1343,-0.0993,-0.0474,-0.0797,0.1235,-0.0049,-0.1264,0.1338,0.0794,-0.1973,0.0365,0.1370,-0.1625,0.0232,0.0475,0.1705,-0.1062,0.0819,-0.1307,-0.0184,-0.1444,0.0134,-0.1509,-0.1874,-0.0826,0.0479,0.1708,0.0411,0.1018,0.1192,-0.1185,0.0309,-0.1410,-0.1051,-0.1491,0.0688,0.0749,-0.1690,-0.1360,-0.1436,0.1567,-0.1553,0.0578,-0.0253,0.1349,0.0344,-0.0000,-0.0268,0.1990,0.1249,-0.1623,0.1765,-0.0806,-0.0077,0.1247,-0


weights in FF Parameter containing:
tensor([[ 0.0635,  0.0722, -0.0308,  ...,  0.0387, -0.0849, -0.0559],
        [-0.0515, -0.0305,  0.0667,  ...,  0.0706, -0.0608,  0.0452],
        [ 0.0084,  0.0693, -0.0299,  ..., -0.0136,  0.0233, -0.0090],
        ...,
        [ 0.0102,  0.0252, -0.0889,  ..., -0.0832, -0.0175,  0.0433],
        [ 0.0219,  0.0867, -0.0955,  ...,  0.0906,  0.0600,  0.0881],
        [-0.0927, -0.0617,  0.0485,  ...,  0.0451,  0.0132, -0.0919]],
       requires_grad=True) 
 bias in FF Parameter containing:
tensor([ 0.0397,  0.0397,  0.0051, -0.0608, -0.0045, -0.0397, -0.0020, -0.0122,
         0.0214,  0.0274,  0.0211, -0.0360,  0.0258, -0.0475, -0.0630, -0.0588,
        -0.0664,  0.0107, -0.0415,  0.0883, -0.0570, -0.0352,  0.0816, -0.0850],
       requires_grad=True)
==============keys============
 tensor([[[[-0.4761,  0.4880, -0.1828],
          [ 0.2723, -0.8745, -0.4046],
          [ 0.0888, -0.3109,  0.4978],
          [-0.4513,  0.6958,  0.6074],
          [

 tensor([[[ 0.4890, -0.0038,  0.0875, -0.3115,  0.4164, -0.0237,  0.2836,
          -0.4313,  0.2050,  0.2013,  0.2665,  0.1229, -0.3958, -0.4174,
          -0.2322, -0.2754,  0.7486, -0.0969,  0.5718, -0.2140,  0.5813,
          -0.2010,  0.0215,  0.1697],
         [ 0.4982, -0.0010,  0.0855, -0.3038,  0.4062, -0.0261,  0.2861,
          -0.4275,  0.2054,  0.1926,  0.2726,  0.1265, -0.3977, -0.4122,
          -0.2376, -0.2745,  0.7415, -0.1007,  0.5690, -0.2058,  0.5728,
          -0.1926,  0.0121,  0.1665]]], grad_fn=<AddBackward0>)
row 1 of all heads concatenated {0.4890,-0.0038,0.0875,-0.3115,0.4164,-0.0237,0.2836,-0.4313,0.2050,0.2013,0.2665,0.1229,-0.3958,-0.4174,-0.2322,-0.2754,0.7486,-0.0969,0.5718,-0.2140,0.5813,-0.2010,0.0215,0.1697,};
row 2 of all heads concatenated {0.4982,-0.0010,0.0855,-0.3038,0.4062,-0.0261,0.2861,-0.4275,0.2054,0.1926,0.2726,0.1265,-0.3977,-0.4122,-0.2376,-0.2745,0.7415,-0.1007,0.5690,-0.2058,0.5728,-0.1926,0.0121,0.1665,};
};
torch.Size([1, 2, 24])
tor

Weights for lin layer [24x24] = { -0.0412,0.1428,-0.1716,0.0934,0.1055,0.1184,0.0720,-0.0174,0.1654,0.1334,0.0662,0.0878,-0.0118,-0.0849,-0.0097,0.0160,-0.1127,0.2037,0.0647,-0.1252,-0.1158,-0.1308,0.1096,0.0985,-0.1632,0.1272,0.2034,-0.0502,-0.1032,-0.0446,0.1098,0.0237,0.0119,-0.0985,0.1354,-0.1014,-0.0581,0.2023,0.0696,-0.1678,0.0239,0.0847,-0.0658,-0.1127,0.1248,-0.1891,0.1686,0.0116,-0.0886,-0.1969,-0.0090,-0.0980,0.0729,-0.0377,0.0301,-0.1513,-0.1770,0.1423,0.0339,0.0661,-0.1945,0.1451,0.2013,-0.0717,-0.1728,-0.1000,0.1828,-0.1633,0.1349,-0.0192,-0.1413,0.1365,-0.0480,-0.1460,0.0757,0.0120,0.1935,0.1822,0.0061,-0.1785,0.0869,0.1361,0.0011,-0.0805,-0.0911,0.1770,0.1589,0.1435,-0.1893,0.0197,-0.1923,-0.0159,0.0161,0.0587,0.0297,0.0432,-0.1624,0.1634,-0.0778,0.1308,0.1440,-0.1197,0.1353,0.0210,-0.0189,-0.1051,0.1540,0.0481,0.1993,-0.1036,-0.0440,0.0031,0.1557,-0.0389,0.1638,0.1699,-0.0476,-0.0151,0.0874,0.0394,0.1675,0.0200,0.1626,-0.1427,-0.1692,0.0017,-0.1609,0.1428,0.0536,-0.0692


weights in FF Parameter containing:
tensor([[ 0.0892,  0.0419, -0.0091,  ..., -0.0829,  0.0078,  0.0401],
        [-0.0359, -0.0257, -0.0731,  ..., -0.0276,  0.0077,  0.0475],
        [-0.0817,  0.0938,  0.0467,  ..., -0.1015,  0.0273, -0.0225],
        ...,
        [ 0.0566,  0.0582,  0.0843,  ...,  0.0306, -0.0305, -0.0061],
        [ 0.0746, -0.0357, -0.0064,  ..., -0.0147, -0.0946,  0.0343],
        [-0.0583,  0.0699, -0.0766,  ...,  0.0526, -0.0954,  0.0130]],
       requires_grad=True) 
 bias in FF Parameter containing:
tensor([ 0.0878,  0.0796, -0.0778, -0.0381, -0.0021, -0.0811,  0.0758,  0.0035,
         0.0891,  0.0746,  0.0823,  0.0316,  0.0152,  0.0456,  0.0131,  0.0007,
        -0.0590,  0.0915, -0.0266,  0.0586, -0.0134,  0.0857,  0.0048, -0.0902],
       requires_grad=True)
torch.Size([1, 2, 10])
tensor([[[ 0.0208,  1.1725,  0.7405,  0.1269,  0.3898,  0.6403,  0.6254,
           0.0852,  1.0949, -0.4497],
         [ 0.3994,  0.9159,  0.0779,  0.0377,  0.1246,  0.4850,  

In [7]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters())

In [8]:
#torch.save(model.state_dict(), "txmodel.pt")

In [9]:
#model.state_dict()

In [10]:
#print(f'layer {layerCount+1}:')